In [31]:
import pandas as pd
from sqlalchemy import create_engine

from flask import request, Flask, render_template
from flask_sqlalchemy import SQLAlchemy
from io import TextIOWrapper
import csv

engine = create_engine('sqlite://', echo=False)

In [21]:
data = {'drug' : ['Tylenol', 'Cialis', 'Ibuprofen', 'Benadril', 'Claritin'],
        'condition' : ['pain', 'ed', 'pain', 'allergy', 'allergy'],
       'headache' : [2, 4, 0, 1, 0],
       'nausea' : [0, 1, 1, 1, 0],
       'dizzy' : [0, 2, 1, 1, 0],
       'death' : [1, 0, 1, 2, 20]}

df = pd.DataFrame(data, columns = data.keys())

df = df.set_index('drug')

In [22]:
df.to_sql('reactions', con=engine, if_exists = 'replace')

In [23]:
engine.execute("SELECT * FROM reactions").fetchall()

[('Tylenol', 'pain', 2, 0, 0, 1),
 ('Cialis', 'ed', 4, 1, 2, 0),
 ('Ibuprofen', 'pain', 0, 1, 1, 1),
 ('Benadril', 'allergy', 1, 1, 1, 2),
 ('Claritin', 'allergy', 0, 0, 0, 20)]

In [29]:
print(df)

          condition  headache  nausea  dizzy  death
drug                                               
Tylenol        pain         2       0      0      1
Cialis           ed         4       1      2      0
Ibuprofen      pain         0       1      1      1
Benadril    allergy         1       1      1      2
Claritin    allergy         0       0      0     20


In [30]:
df.to_csv('../raw_data/dummydata.csv')

In [2]:
datatest = pd.read_csv('../raw_data/dummydata.csv')

In [4]:
html_block = pd.read_csv('../raw_data/dummydata.csv').to_html()

In [5]:
html_block

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>drug</th>\n      <th>condition</th>\n      <th>headache</th>\n      <th>nausea</th>\n      <th>dizzy</th>\n      <th>death</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Tylenol</td>\n      <td>pain</td>\n      <td>2</td>\n      <td>0</td>\n      <td>0</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Cialis</td>\n      <td>ed</td>\n      <td>4</td>\n      <td>1</td>\n      <td>2</td>\n      <td>0</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Ibuprofen</td>\n      <td>pain</td>\n      <td>0</td>\n      <td>1</td>\n      <td>1</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Benadril</td>\n      <td>allergy</td>\n      <td>1</td>\n      <td>1</td>\n      <td>1</td>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Claritin</td>\n      <td>allergy</td>\n      <td>0</td>\n 

In [15]:
#render dataframe as html
html = df.to_html()

#write html to file
text_file = open("index.html", "w")
text_file.write(html)
text_file.close()

In [28]:
pd.read_sql_table('reactions', con = engine)

,drug,condition,headache,nausea,dizzy,death
0,Tylenol,pain,2,0,0,1
1,Cialis,ed,4,1,2,0
2,Ibuprofen,pain,0,1,1,1
3,Benadril,allergy,1,1,1,2
4,Claritin,allergy,0,0,0,20


In [32]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///test.db'
db = SQLAlchemy(app)
csv_file = request.files['../raw_data/dummydata.csv']
csv_file = TextIOWrapper(csv_file, encoding='utf-8')
csv_reader = csv.reader('../raw_data/dummydata.csv', delimiter=',')
db.create_all()

class Drug(db.Model):
    id = db.Column(db.Integer, primary_key = True)
    drugname = db.Column(db.String(30), unique=True)
#     condition = db.Column(db.String(20), unique=False)
#     headache = db.Column(db.Integer, unique=False)
#     nausea = db.Column(db.Integer, unique=False)
#     dizzy = db.Column(db.Integer, unique=False)
#     death = db.Column(db.Integer, unique=False)
    def __repr__(self):
        return '<Drug %r>' % self.drugname

# Drug.createall()
for row in csv_reader:
    print(row)
    drug = Drug(drugname=row[0])
    print(drug)
    db.session.add(drug)
    db.session.commit()
# fulldata = db.session.query(Drug).all()

c:\users\pefle\.venvs\adverse_drug_reactions\lib\site-packages\flask_sqlalchemy\__init__.py:834: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request.  Consult the documentation on testing for
information about how to avoid this problem.